In [44]:
import os
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta, date

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,  # Используем INFO для менее подробного логирования
    format='%(asctime)s - %(levelверh)s - %(message)s',
    handlers=[
        logging.FileHandler("algorithm_one.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)

def get_initial_date():
    """
    Получает самую раннюю дату из таблицы OlapCube_VNV.
    """
    try:
        query = f"""
        SELECT MIN(Dates) AS initial_date
        FROM {database_name}.OlapCube_VNV
        """
        result = client.execute(query)
        initial_date = result[0][0]
        logger.info(f"Начальная дата в OlapCube_VNV: {initial_date}")
        return initial_date
    except Exception as e:
        logger.error(f"Ошибка при получении начальной даты: {e}", exc_info=True)
        return None

def get_target_date(initial_date):
    """
    Вычисляет target_date = initial_date + 1 день.
    """
    try:
        if isinstance(initial_date, datetime):
            initial_date_date = initial_date.date()
        elif isinstance(initial_date, date):
            initial_date_date = initial_date
        elif isinstance(initial_date, str):
            initial_date_date = datetime.strptime(initial_date, "%Y-%m-%d").date()
        else:
            logger.error(f"Неизвестный формат initial_date: {initial_date}")
            return None
        target_date = initial_date_date + timedelta(days=1)
        logger.info(f"target_date: {target_date}")
        return target_date
    except Exception as e:
        logger.error(f"Ошибка при вычислении target_date: {e}", exc_info=True)
        return None

def fetch_previous_and_target_data(target_date):
    """
    Извлекает данные для target_date -1 и target_date из OlapCube_VNV.
    Также извлекает текущее значение Status_P для target_date.
    """
    try:
        previous_date = target_date - timedelta(days=1)
        logger.info(f"Обработка предыдущей даты: {previous_date}")
        logger.info(f"Обработка target_date: {target_date}")

        # Извлечение данных за предыдущую дату
        query_prev = f"""
        SELECT 
            serialno, Status, sne, ppr, ll, oh, BR, daily_flight_hours, repair_days, RepairTime
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{previous_date}'
        """
        prev_data = client.execute(query_prev)
        logger.info(f"Извлечено {len(prev_data)} записей за предыдущую дату {previous_date}.")

        # Извлечение данных за target_date, включая текущее значение Status_P
        query_target = f"""
        SELECT 
            serialno, Status_P, ll, oh, BR, daily_flight_hours
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        """
        target_data = client.execute(query_target)
        logger.info(f"Извлечено {len(target_data)} записей за target_date {target_date}.")

        # Преобразование данных в словари для быстрого доступа
        prev_dict = {row[0]: row for row in prev_data}
        target_dict = {row[0]: row for row in target_data}

        return prev_dict, target_dict
    except Exception as e:
        logger.error(f"Ошибка при извлечении данных: {e}", exc_info=True)
        return None, None

def calculate_status_p(prev_dict, target_dict, target_date):
    """
    Рассчитывает Status_P для target_date на основе предыдущих данных и условий.
    Возвращает список обновлений: (Status_P, serialno, target_date)
    """
    status_updates = []
    unresolved = []
    changed_serialnos = []

    for serialno, target_row in target_dict.items():
        target_status_p = target_row[1]  # Текущее значение Status_P
        target_ll = target_row[2]
        target_oh = target_row[3]
        target_BR = target_row[4]
        target_daily_flight_hours = target_row[5]

        prev_row = prev_dict.get(serialno, None)
        if not prev_row:
            logger.warning(f"Не найдены данные за предыдущую дату для serialno: {serialno}")
            unresolved.append(serialno)
            continue

        prev_status = prev_row[1]
        prev_sne = prev_row[2]
        prev_ppr = prev_row[3]
        prev_ll = prev_row[4]
        prev_oh = prev_row[5]
        prev_BR = prev_row[6]
        prev_daily_flight_hours = prev_row[7]
        prev_repair_days = prev_row[8]
        repair_time = prev_row[9]

        # Проверка на None
        if None in [prev_sne, prev_ppr, target_ll, target_oh, target_BR, target_daily_flight_hours]:
            logger.warning(f"Пропускаем запись с serialno: {serialno} из-за отсутствия данных.")
            unresolved.append(serialno)
            continue

        # Приведение типов
        try:
            prev_sne = float(prev_sne)
            prev_ppr = float(prev_ppr)
            target_ll = float(target_ll)
            target_oh = float(target_oh)
            target_BR = float(target_BR)
            target_daily_flight_hours = float(target_daily_flight_hours)
            prev_repair_days = float(prev_repair_days) if prev_repair_days is not None else 0
            repair_time = float(repair_time) if repair_time is not None else 0
        except ValueError as ve:
            logger.warning(f"Некорректные числовые значения для serialno: {serialno}. Ошибка: {ve}")
            unresolved.append(serialno)
            continue

        # Инициализация Status_P
        status_p = None

        # 1. Эксплуатация
        if prev_status == "Эксплуатация":
            condition_sne = prev_sne < (target_ll - target_daily_flight_hours)
            condition_ppr = prev_ppr < (target_oh - target_daily_flight_hours)

            if condition_sne and condition_ppr:
                status_p = "Эксплуатация"
            else:
                if prev_sne < target_BR:
                    status_p = "Ремонт"
                else:
                    status_p = "Хранение"

        # 2. Ремонт
        elif prev_status == "Ремонт":
            if prev_repair_days < repair_time:
                status_p = "Ремонт"
            else:
                status_p = "Исправен"

        # 3. Исправен, Неактивно, Хранение
        elif prev_status in ["Исправен", "Неактивно", "Хранение"]:
            status_p = prev_status

        else:
            unresolved.append(serialno)
            logger.warning(f"Неизвестный Status_P для serialno: {serialno} с предыдущим статусом: {prev_status}")
            continue

        if status_p:
            # Добавляем все статусы для обновления, независимо от изменений
            status_updates.append((status_p, serialno, target_date))
            # Логируем только изменения статусов
            old_status_p_clean = prev_status.strip().lower() if isinstance(prev_status, str) else prev_status
            new_status_p_clean = status_p.strip().lower() if isinstance(status_p, str) else status_p
            if old_status_p_clean != new_status_p_clean:
                changed_serialnos.append(serialno)
                logger.info(f"serialno: {serialno} - Status_P изменился с '{prev_status}' на '{status_p}'")
        else:
            unresolved.append(serialno)
            logger.warning(f"Не удалось определить Status_P для serialno: {serialno}")

    if unresolved:
        logger.error(f"Не удалось определить Status_P для следующих serialno: {unresolved}")

    logger.info(f"Определено {len(status_updates)} записей для обновления Status_P.")
    if changed_serialnos:
        logger.info(f"Изменились статусы для следующих serialno: {', '.join(changed_serialnos)}")
        logger.info(f"Общее количество измененных serialno: {len(changed_serialnos)}")
    else:
        logger.info("Изменения статусов не зафиксированы.")

    return status_updates

def update_status_p(status_updates, target_date):
    """
    Обновляет поле Status_P в таблице OlapCube_VNV для target_date.
    """
    try:
        for status_p, serialno, date in status_updates:
            update_query = f"""
            ALTER TABLE {database_name}.OlapCube_VNV 
            UPDATE Status_P = %(status_p)s 
            WHERE serialno = %(serialno)s AND Dates = %(date)s
            """
            params = {
                'status_p': status_p,
                'serialno': serialno,
                'date': date
            }
            try:
                client.execute(update_query, params)
            except Exception as e_inner:
                logger.error(f"Ошибка при обновлении Status_P для serialno: {serialno}, Dates: {date}. Ошибка: {e_inner}", exc_info=True)
                continue  # Продолжаем обработку остальных записей
        logger.info(f"Обновлено {len(status_updates)} записей в OlapCube_VNV для даты {target_date}.")
    except Exception as e:
        logger.error(f"Ошибка при обновлении Status_P: {e}", exc_info=True)

def main():
    # Шаг 1: Получение начальной даты
    initial_date = get_initial_date()
    if not initial_date:
        logger.error("Не удалось определить начальную дату. Завершаем выполнение.")
        return

    # Шаг 2: Вычисление target_date
    target_date = get_target_date(initial_date)
    if not target_date:
        logger.error("Не удалось определить target_date. Завершаем выполнение.")
        return

    # Шаг 3: Извлечение данных за предыдущую и target_date
    prev_dict, target_dict = fetch_previous_and_target_data(target_date)
    if not prev_dict or not target_dict:
        logger.error("Не удалось извлечь необходимые данные. Завершаем выполнение.")
        return

    # Шаг 4: Расчет Status_P
    status_updates = calculate_status_p(prev_dict, target_dict, target_date)
    if not status_updates:
        logger.info("Нет записей для обновления Status_P.")
        return

    # Шаг 5: Обновление Status_P в таблице
    update_status_p(status_updates, target_date)

if __name__ == "__main__":
    main()


2024-12-02 09:32:17,927 - INFO - Начальная дата в OlapCube_VNV: 2024-11-25
2024-12-02 09:32:17,929 - INFO - target_date: 2024-11-26
2024-12-02 09:32:17,930 - INFO - Обработка предыдущей даты: 2024-11-25
2024-12-02 09:32:17,931 - INFO - Обработка target_date: 2024-11-26
2024-12-02 09:32:18,040 - INFO - Извлечено 420 записей за предыдущую дату 2024-11-25.
2024-12-02 09:32:18,098 - INFO - Извлечено 420 записей за target_date 2024-11-26.
2024-12-02 09:32:18,100 - INFO - serialno: 1228С101 - Status_P изменился с 'Ремонт' на 'Исправен'
2024-12-02 09:32:18,101 - INFO - serialno: 2228С78 - Status_P изменился с 'Ремонт' на 'Исправен'
2024-12-02 09:32:18,103 - INFO - serialno: С5381040 - Status_P изменился с 'Ремонт' на 'Исправен'
2024-12-02 09:32:18,104 - INFO - serialno: С6384678 - Status_P изменился с 'Ремонт' на 'Исправен'
2024-12-02 09:32:18,105 - INFO - serialno: СВ78В615 - Status_P изменился с 'Ремонт' на 'Исправен'
2024-12-02 09:32:18,106 - INFO - Определено 420 записей для обновления St